This script recorded my pipeline to create the study 'mouse_VISp_L5_128ch'.

In [1]:
from utils import *
from simulate.simulate import *
from compare.compare import *

import numpy as np
import pickle as pkl
import shutil

import MEArec as mr
import MEAutility as mu

First retrieve Allen cell models in `cell_models` folder. These models were unzipped and manually categorized into 3 subfolders: `aspiny`, `sparsely_spiny`, `spiny`.

Then create a new study.

In [3]:
study_name = 'mouse_VISp_L5_128ch'
study_dir = './studies'
study_path = create_empty_study(study_name,study_dir)

Then generate templates. Set up the params for generating templates.

In [4]:
# get default
template_params = mr.get_default_templates_params()

############### Intracellular simulation settings

template_params['sim_time'] = 1 # float, intracellular simulation time (sec)
template_params['target_spikes'] = [3,50] # int, min-max number of spikes in sim_time
template_params['cut_out'] = [2,5] # float, pre/post peak cut_out (msec)
template_params['dt'] = 0.03125 # float, time step (msec), default in 32 kHz
template_params['delay'] = 10 # float, simulation delay (msec)
template_params['weights'] = [0.25,1.75] # float, weights to multiply stimulus amp when spikes number above/below targets

############### Extracellular simulation settings

template_params['rot'] = '3drot' # cell model rotation, ['norot','xrot','yrot','zrot','physrot','3drot']
print(mu.return_mea_list())
template_params['probe'] = 'Neuropixels2-128' # probe, can customize
template_params['ncontacts'] = 10 # int, number of contacts per recording site
template_params['overhang'] = 30 # float, extension (um) beyond MEA boundraies for neuron location, when null limits
template_params['xlim'] = [10,80] # float, limits in x-axis
template_params['ylim'] = None # float, limits in y-axis
template_params['zlim'] = None # float, limits in z-axis
template_params['min_amp'] = 30 # float, min template amplitude
template_params['n'] = 10 # int, number of spike per cell model
template_params['n_overlap_pairs'] = None # number of spatial overlapping templates
template_params['seed'] = 0 # random seed for positions and rotations

#### drifting
template_params['drifting'] = True # bool, turn on drifting simulation
template_params['max_drift'] = 100 # float, max distance for drifting
template_params['min_drift'] = 30 # float, min distance for drifting
template_params['drift_steps'] = 30 # int, number of drift steps
template_params['drift_xlim'] = [-10,10] # float, limits for drift in x-axis
template_params['drift_ylim'] = [-10,10] # float, limits for drift in y-axis
template_params['drift_zlim'] = [20,80] # float, limits for drift in z-axis

#####################################

['Neuronexus-32', 'Neuropixels-128', 'Neuropixels-24', 'Neuropixels-384', 'Neuropixels-64', 'Neuropixels2-128', 'Neuropixels2-24', 'Neuropixels2-384', 'Neuropixels2-64', 'Neuroseeker-128', 'SqMEA-10-15', 'SqMEA-15-10', 'SqMEA-5-30', 'SqMEA-6-25', 'SqMEA-7-20', 'monotrode', 'tetrode', 'tetrode-mea-d', 'tetrode-mea-l', 'tetrode-mea-s']


Generate templates.

Copy probe files from `MEAutility.electrodes`

In [5]:
template_path = create_empty_templates(study_path,'template_drift')

cell_model_path = Path('./cell_models/mouse_VISp_L5/')
cell_sim_model_path = study_path / 'cell_sim_model'
template_path = template_path

# save template_params
with open((template_path / 'template_params.pkl').as_posix(), 'wb') as f:
    pkl.dump(template_params,f)

generate_template(template_params,cell_model_path,cell_sim_model_path,template_path)

{'beta_distr_params': [1.5, 5],
 'check_for_drift_amp': False,
 'cut_out': [2, 5],
 'delay': 10,
 'drift_steps': 30,
 'drift_within_bounds': False,
 'drift_xlim': [-10, 10],
 'drift_ylim': [-10, 10],
 'drift_zlim': [20, 80],
 'drifting': True,
 'dt': 0.03125,
 'max_drift': 100,
 'max_iterations': 1000,
 'min_amp': 30,
 'min_drift': 30,
 'n': 10,
 'n_overlap_pairs': None,
 'ncontacts': 10,
 'offset': 0,
 'overhang': 30,
 'probe': 'Neuropixels2-128',
 'rot': '3drot',
 'seed': 0,
 'sim_time': 1,
 'target_spikes': [3, 50],
 'timeout': None,
 'weights': [0.25, 1.75],
 'x_distr': 'uniform',
 'xlim': [10, 80],
 'ylim': None,
 'zlim': None}
[WindowsPath('cell_models/mouse_VISp_L5/spiny/neuronal_model_471087975'), WindowsPath('cell_models/mouse_VISp_L5/spiny/neuronal_model_472306460'), WindowsPath('cell_models/mouse_VISp_L5/spiny/neuronal_model_473863510'), WindowsPath('cell_models/mouse_VISp_L5/spiny/neuronal_model_473863578'), WindowsPath('cell_models/mouse_VISp_L5/spiny/neuronal_model_473871

Then generate recordings using this template. Set up the parameters for generating recordings.

In [6]:
### rec_params with drifting

# get default
rec_params = mr.get_default_recordings_params()

rec_params['cell_types'] = {'excitatory': ['spiny','sparsely_spiny'], 'inhibitory': ['aspiny']}

rec_params['spiketrains']['n_exc'] = 40 # int, number of excitatory cells, less than number of templates
rec_params['spiketrains']['n_inh'] = 20 # int, number of inhibitory cells
rec_params['spiketrains']['f_exc'] = 5 # float, average firing rate of exc cells (Hz)
rec_params['spiketrains']['f_inh'] = 15 # float, average firing rate of inh cells (Hz)
rec_params['spiketrains']['st_exc'] = 1 # float, firing rate standard deviation of exc cells
rec_params['spiketrains']['st_inh'] = 3 # float, firing rate standard deviation of inh cells
rec_params['spiketrains']['min_rate'] = 0.5 # float, min firing rate (Hz)
rec_params['spiketrains']['ref_per'] = 2 # float, refractory period (msec)
rec_params['spiketrains']['process'] = 'poisson' # process of spike train simulation, ['poisson','gamma']
rec_params['spiketrains']['gamma_shape'] = 2 # gamma shape for gamma process
rec_params['spiketrains']['duration'] = 30 # float, recording time

rec_params['templates']['min_dist'] = 25 # float, min distance between cells
rec_params['templates']['min_amp'] = 50 # float, min spike amplitude (uV)
rec_params['templates']['max_amp'] = 500 # float, max spike amplitude (uV)
rec_params['templates']['xlim'] = None # limits of neurons in x-axis
rec_params['templates']['ylim'] = None # limits of neurons in y-axis
rec_params['templates']['zlim'] = None # limits of neurons in z-axis
rec_params['templates']['overlap_threshold'] = 0.9 # float, threshold to consider 2 templates spatially overlapping
rec_params['templates']['n_jitters'] = 10 # int, number of temporal jittered copies for each template (temporal shift within one sampling period)
rec_params['templates']['upsample'] = 8 # int, upsampling factor to extract jittered copies
rec_params['templates']['pad_len'] = [3,3] # float, pre/post time padding of templates (msec)

rec_params['recordings']['fs'] = None # if None, computed from templates
rec_params['recordings']['dtype'] = 'float32'
rec_params['recordings']['overlap'] = True # compute temporal and spatial overlap for each spike
rec_params['recordings']['extract_waveforms'] = True # extract waveforms from recordings
rec_params['recordings']['sync_rate'] = 0.05 # sync rate 0-1 for spatially overlapping templates
rec_params['recordings']['sync_jitt'] = 1 # float, jitter for added sync spikes (msec)
rec_params['recordings']['modulation'] = 'electrode' # type of spike modulation ['none','template','electrode'] 'template': same value on each electrode; 'electrode': different value on each electrode
rec_params['recordings']['sdrand'] = 0.05 # float, standard deviation of Guassian modulation, when bursting is False

rec_params['recordings']['bursting'] = True # modulate spikes in amplitude depending on ISI
rec_params['recordings']['exp_decay'] = 0.1 # float, when bursting is True, decay in amplitude between consecutive spikes
rec_params['recordings']['n_burst_spikes'] = 10 # int, max bursting consecutive spikes
rec_params['recordings']['max_burst_duration'] = 100 # float, max burst modulation time (msec)
rec_params['recordings']['shape_mod'] = True # stretch spike shapes with sigmoid transform
rec_params['recordings']['shape_stretch'] = 30 # float, amount of stretch for shape modulation
rec_params['recordings']['n_bursting'] = None # int, number of bursting cells, if None, all are bursting
rec_params['recordings']['chunk_duration'] = 20 # sec, processing chunk

rec_params['recordings']['noise_mode'] = 'distance-correlated' # ['uncorrelated','distance-correlated','far-neurons']
rec_params['recordings']['noise_level'] = 10 # float, noise standard deviation (uV)
rec_params['recordings']['noise_color'] = True # have colored noise
rec_params['recordings']['noise_half_distance'] = 30 # distance (um) between electrodes for which correlation is 0.5
rec_params['recordings']['color_peak'] = 300 # peak/cutoff frequency for colored noise (Hz)
rec_params['recordings']['color_q'] = 2 # quality factor for colored noise
rec_params['recordings']['color_noise_floor'] = 0.5 # percent of addictive noise for colored noise
rec_params['recordings']['far_neurons_n'] = 300 # number of far neurons to be simulated
rec_params['recordings']['far_neurons_max_amp'] = 10 # max amplitude of far neurons
rec_params['recordings']['far_neurons_noise_floor'] = 0.5 # percent of addictive noise for far-neurons noise
rec_params['recordings']['far_neurons_exc_inh_ratio'] = 0.8 # e/i noisy neuron ratio 0-1

rec_params['recordings']['filter'] = False # filter the recording
rec_params['recordings']['filter_cutoff'] = [300,6000]
rec_params['recordings']['filter_order'] = 3

rec_params['recordings']['drifting'] = True # simulate drifting recordings
rec_params['recordings']['n_drifting'] = None # number of drifting cells, if None, all are drifting
rec_params['recordings']['perferred_dir'] = [0,0,1] # drifting direction, positive z-axis
rec_params['recordings']['angle_tol'] = 90 # tolerance for direction (degree)
rec_params['recordings']['slow_drift_velocity'] = 5 # slow drift (um/min)
rec_params['recordings']['fast_drift_period'] = 20 # period between fast drift (sec)
rec_params['recordings']['fast_drift_max_jump'] = 20 # max amplitude jump for fast drifts (uV)
rec_params['recordings']['fast_drift_min_jump'] = 5 # min amplitude jump for fast drift
rec_params['recordings']['t_start_drift'] = 0 # when start drifting (sec)

rec_params['seeds']['spiketrains'] = None
rec_params['seeds']['templates'] = None
rec_params['seeds']['convolution'] = None
rec_params['seeds']['noise'] = None


Generate recordings.

In [7]:
recording_path = create_empty_recordings(study_path,'recording_drift')

# save template_params
with open((recording_path / 'recording_params.pkl').as_posix(), 'wb') as f:
    pkl.dump(rec_params,f)

generate_recording(rec_params, template_path, recording_path)

{'cell_types': {'excitatory': ['spiny', 'sparsely_spiny'],
                'inhibitory': ['aspiny']},
 'recordings': {'adc_bit_depth': None,
                'angle_tol': 90,
                'bursting': True,
                'bursting_units': None,
                'chunk_duration': 20,
                'color_noise_floor': 0.5,
                'color_peak': 300,
                'color_q': 2,
                'drift_fs': 100,
                'drift_mode_probe': 'rigid',
                'drift_mode_speed': 'slow',
                'drifting': True,
                'dtype': 'float32',
                'exp_decay': 0.1,
                'extract_waveforms': True,
                'far_neurons_exc_inh_ratio': 0.8,
                'far_neurons_max_amp': 10,
                'far_neurons_n': 300,
                'far_neurons_noise_floor': 0.5,
                'fast_drift_max_jump': 20,
                'fast_drift_min_jump': 5,
                'fast_drift_period': 20,
                'filter': False,


g:\anaconda3\lib\site-packages\MEArec\generators\recgensteps.py:264: RuntimeWarning: covariance is not positive-semidefinite.
  additive_noise = noise_level * np.random.multivariate_normal(np.zeros(n_elec), cov_dist,
g:\anaconda3\lib\site-packages\MEArec\generators\recgensteps.py:271: RuntimeWarning: covariance is not positive-semidefinite.
  np.random.multivariate_normal(np.zeros(n_elec), cov_dist,


Extracting spike waveforms
Elapsed time:  2300.9501077000004
Impossible to delete temp file: TXadB_mearec_tmp_file_recordings.raw Error [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'TXadB_mearec_tmp_file_recordings.raw'
Impossible to delete temp file: TXadB_mearec_tmp_file_spike_traces.raw Error [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'TXadB_mearec_tmp_file_spike_traces.raw'
Deleted TXadB_templates_pad.raw
Impossible to delete temp file: TXadB_templates_resample.raw Error [WinError 2] 系统找不到指定的文件。: 'TXadB_templates_resample.raw'
Deleted TXadB_templates_jitter.raw
Deleted TXadB_mearec_tmp_noise_file.raw
